## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate  source  \
0  2025-10-21-18-54-38 +0000     nyt   
1  2025-10-21-18-49-36 +0000  nypost   
2  2025-10-21-18-48-17 +0000     cbc   
3  2025-10-21-18-40-20 +0000     nyt   
4  2025-10-21-18-39-57 +0000     nyt   

                                               title  \
0  Vance Lands in Israel as U.S. Tries to Shore U...   
1  HBO Max prices going up again — here’s how muc...   
2  Liberal House leader ‘starting to worry’ about...   
3  North Carolina Senate Approves New Map in Effo...   
4      2025 General Election Guide for This November   

                                                link  
0  https://www.nytimes.com/2025/10/21/world/middl...  
1  https://nypost.com/2025/10/21/media/warner-bro...  
2  https://www.cbc.ca/news/politics/liberal-budge...  
3  https://www.nytimes.com/2025/10/21/us/north-ca...  
4  https://www.nytimes.com/2025/10/20/us/politics...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
44   trump     59
29     new     16
18   house     16
5       up     16
135  white     14

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate   source  \
289  2025-10-20-22-36-00 +0000      wsj   
337  2025-10-20-20-06-32 +0000  latimes   
36   2025-10-21-17-27-00 +0000      wsj   
72   2025-10-21-16-24-49 +0000     wapo   
278  2025-10-20-23-05-53 +0000   nypost   

                                                 title  \
289  President Trump’s $100,000 H-1B visa fee will ...   
337  Trump again suggests sending troops to San Fra...   
36   Key Senate Republicans are calling for the wit...   
72   White House says no Putin-Trump summit anytime...   
278  Trump can deploy National Guard to Portland, O...   

                                                  link  score  
289  https://www.wsj.com/politics/policy/u-s-narrow...    139  
337  https://www.latimes.com/california/story/2025-...    137  
36   https://www.wsj.com/politics/policy/paul-ingra...    134  
72   https://www.washingtonpost.com/world/2025/10/2...    114  
278  https://nypost.com/2025/10/20/us-news/9th-circ...    111

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate       source  \
289    139  2025-10-20-22-36-00 +0000          wsj   
0       76  2025-10-21-18-54-38 +0000          nyt   
36      67  2025-10-21-17-27-00 +0000          wsj   
235     53  2025-10-21-04-53-16 +0000         wapo   
278     52  2025-10-20-23-05-53 +0000       nypost   
283     41  2025-10-20-23-00-00 +0000          wsj   
54      41  2025-10-21-17-01-16 +0000  startribune   
129     41  2025-10-21-13-25-26 +0000       nypost   
123     38  2025-10-21-13-56-46 +0000       nypost   
23      33  2025-10-21-17-44-24 +0000       nypost   

                                                 title  \
289  President Trump’s $100,000 H-1B visa fee will ...   
0    Vance Lands in Israel as U.S. Tries to Shore U...   
36   Key Senate Republicans are calling for the wit...   
235  Japan elects first female leader, a hawk who p...   
278  Trump can deploy National Guard to Portland, O...   
283  Business and political leaders have intensifie...   
54   Neal: Ex-Twins manager Rocco Baldelli looks ba...   
129  Former French President Sarkozy begins 5-year ...   
123  Warner Bros. Discovery says it’s open to a sal...   
23   Hamas to release bodies of two more dead hosta...   

                                                  link  
289  https://www.wsj.com/politics/policy/u-s-narrow...  
0    https://www.nytimes.com/2025/10/21/world/middl...  
36   https://www.wsj.com/politics/policy/paul-ingra...  
235  https://www.washingtonpost.com/world/2025/10/2...  
278  https://nypost.com/2025/10/20/us-news/9th-circ...  
283  https://www.wsj.com/politics/elections/nyc-may...  
54   https://www.startribune.com/neal-ex-twins-mana...  
129  https://nypost.com/2025/10/21/world-news/forme...  
123  https://nypost.com/2025/10/21/business/warner-...  
23   https://nypost.com/2025/10/21/world-news/hamas...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
